In [1]:
import os
import numpy as np
import cv2

In [2]:
forg_folder = 'CEDAR_database/full_forg/'
org_folder = 'CEDAR_database/full_org/'
all_forg = os.listdir(forg_folder)
all_org = os.listdir(org_folder)

In [3]:
data = []
labels = []
    
for i in all_org:
    curr_img = cv2.imread(org_folder + i, 0)
    curr_img = cv2.resize(curr_img,(240, 160),interpolation=cv2.INTER_AREA)
    if i[10] == '_':
        class_pres = int(i[9:10])
    else:
        class_pres = int(i[9:11])
    labels.append(class_pres)
    data.append(curr_img)
    
for i in all_forg:
    curr_img = cv2.imread(forg_folder + i, 0)
    curr_img = cv2.resize(curr_img,(240, 160),interpolation=cv2.INTER_AREA)
    if i[11] == '_':
        class_pres = int(i[10:11]) + 56
    else:
        class_pres = int(i[10:12]) + 56
    labels.append(class_pres)
    data.append(curr_img)


In [4]:
labels = np.array(labels)
data = np.array(data)

In [5]:
data.shape

(2640, 160, 240)

In [6]:
labels.shape

(2640,)

In [7]:
labels

array([10, 10, 10, ..., 65, 65, 65])

In [8]:
from numpy import newaxis
data = data[:,:,:,newaxis]
data.shape

(2640, 160, 240, 1)

In [9]:
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils

c:\programdata\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [10]:
X_train,X_test,y_train,y_test = train_test_split(data, labels, test_size=0.125)

In [11]:
# normalizing dataset
X_train = X_train / 255
X_test = X_test / 255

In [12]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#checking total classes
y_test.shape[1]

112

In [13]:
X_train.shape[0]

2310

In [14]:
#importing modules for NN creation
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD

In [17]:
def make_model():
    model = Sequential()
    model.add(Conv2D(filters=8,kernel_size=(5,5),input_shape=(160,240,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=16,kernel_size=(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=8,kernel_size=(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=128,activation='relu'))
    model.add(Dense(units=64,activation='relu'))
    model.add(Dense(units=112,activation='softmax'))
    model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [18]:
model = make_model()
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=5)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 2310 samples, validate on 330 samples
Epoch 1/40
2310/2310 [==============================] - 86s - loss: 4.7192 - acc: 0.0061 - val_loss: 4.7203 - val_acc: 0.0000e+00
Epoch 2/40
2310/2310 [==============================] - 32s - loss: 4.7152 - acc: 0.0165 - val_loss: 4.7221 - val_acc: 0.0091
Epoch 3/40
2310/2310 [==============================] - 33s - loss: 4.7039 - acc: 0.0165 - val_loss: 4.7107 - val_acc: 0.0061
Epoch 4/40
2310/2310 [==============================] - 31s - loss: 4.6463 - acc: 0.0320 - val_loss: 4.5783 - val_acc: 0.0394
Epoch 5/40
2310/2310 [==============================] - 34s - loss: 4.5076 - acc: 0.0567 - val_loss: 4.7247 - val_acc: 0.0061
Epoch 6/40
2310/2310 [==============================] - 31s - loss: 4.4901 - acc: 0.0589 - val_loss: 4.7288 - val_acc: 0.0091
Epoch 7/40
2310/2310 [==============================] - 32s - loss: 4.7166 - acc: 0.0082 - val_loss: 4.7246 - val_acc: 0.0030
Epoch 8/40
2310/2310 [==============================] - 31s - loss: